In [3]:
import sys
sys.path.append('../../../')

In [4]:
import numpy as np

from occts.classifiers.deep import InceptionModel, OneClassTrainer
from occts.classifiers.deep_task import OneClassTask
from occts.utils.data import load_ucr_dataset, create_datasets, create_dataloaders

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

from typing import Tuple, Optional

In [5]:
DATASETS = [
    'Yoga', 'WormsTwoClass', 'Wine', 'Wafer', 'TwoLeadECG', 'Strawberry', 'SemgHandGenderCh2', 'BeetleFly',
    'BirdChicken', 'Computers', 'DistalPhalanxOutlineCorrect', 'Earthquakes', 'ECG200', 'ECGFiveDays',
    'FordA', 'FordB', 'HandOutlines', 'ItalyPowerDemand', 'MiddlePhalanxOutlineCorrect', 'Chinatown',
    'FreezerRegularTrain', 'FreezerSmallTrain', 'GunPointAgeSpan', 'GunPointMaleVersusFemale',
    'GunPointOldVersusYoung', 'PowerCons', 'Coffee', 'Ham', 'Herring', 'Lightning2', 'MoteStrain',
    'PhalangesOutlinesCorrect', 'ProximalPhalanxOutlineCorrect', 'ShapeletSim', 'SonyAIBORobotSurface1',
    'SonyAIBORobotSurface2', 'ToeSegmentation1', 'ToeSegmentation2', 'HouseTwenty'
]

In [6]:
dataset_list = []
dataset_label = []
dataset_roc_auc = []
dataset_acc = []

In [8]:
for dataset in DATASETS:
    print(f'Testing with dataset: {dataset}...')
    x_train, x_test, y_train, y_test = load_ucr_dataset(dataset=dataset)

    # Get unique labels to do the one class classification task
    unique_labels = np.unique(y_train)

    for label in unique_labels:
        print(f'\tClassifying the label {label}...')

        train_dataset, _, test_dataset = create_datasets(x_train, x_test, y_train, y_test, label)
        train_loader, _, test_loader = create_dataloaders(train_dataset=train_dataset, test_dataset=test_dataset)

        model = InceptionModel(
            num_blocks=6,
            in_channels=1,
            out_channels=32,
            bottleneck_channels=32,
            kernel_sizes=41,
            use_residuals = True,
            latent_dim = 32
        )
        octask = OneClassTask(model=model, objective='soft-boundary', nu=0.1)

        octask.train(dataloader=train_loader, epochs=150, lr=1e-3, weight_decay=1e-6)
        octask.test(dataloader=test_loader)

        dataset_list.append(dataset)
        dataset_label.append(label)
        dataset_acc.append(octask.trainer.test_acc)
        dataset_roc_auc.append(octask.trainer.test_auc)

        break
    break

Testing with dataset: Yoga...
	Classifying the label 1...
			Test set AUC 0.5180194688439084
			Test set Accuracy 0.4673333333333333
